In [1]:
import numpy as np, pandas as pd
import torch, os, logging

from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

from preprocessing import load_dataset, normalize

# Set up Python logging
logging.basicConfig(level=logging.ERROR)

# Create a log directory with a timestamp to keep different runs separate
logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
os.makedirs(logdir, exist_ok=True)
writer = SummaryWriter(logdir)

pd.set_option('display.precision', 20)

torch.manual_seed(1234)
torch.autograd.set_detect_anomaly(True)

if __name__ == "__main__":

    # Load Data (change path if needed)
    path = ["airFoil2D_SST_31.677_-0.742_2.926_5.236_5.651"]
    train_data, len_list = load_dataset(path, 
                                       n_random_sampling = 0
                                       )

    Uinf, alpha, gamma_1, gamma_2, gamma_3 = float(path[0].split('_')[2]), float(path[0].split('_')[3])*np.pi/180, float(path[0].split('_')[4]), float(path[0].split('_')[5]), float(path[0].split('_')[6])
    print(f"Uinf: {Uinf}, alpha: {alpha}")
    
    u_inlet, v_inlet = np.cos(alpha)*Uinf, np.sin(alpha)*Uinf
    
    df_train_input = pd.DataFrame(train_data[0].x_train, columns=["x", "y", "sdf"])
    df_train_target = pd.DataFrame(train_data[0].y_train, columns=["u", "v", "p", "nut"])
    df_train = pd.concat([df_train_input, df_train_target], axis=1) 
    
    df_freestream = df_train.iloc[:len_list[0],:]
    print(df_freestream)

    df_train_n, mean_variance_dict = normalize(df_train) # [len(freestream_attr), len(aerofoil_attr), len(internal_attr), len(internal_attr_sample)]
    
    df_freestream_n = df_train_n.iloc[:len_list[0],:]
    df_aerofoil_n = df_train_n.iloc[len_list[0]:len_list[0]+len_list[1],:]
    df_box_n = df_train_n.iloc[len_list[0]:len_list[0]+len_list[1]+len_list[2],:]
        
    print("Datasets Loaded.")

100%|██████████| 1/1 [00:00<00:00,  4.12it/s]

Uinf: 31.677, alpha: -0.012950343049797924
                            x                     y                   sdf  \
0      0.42432203888893127441  200.0000000000000000  199.9449615478515625   
1      0.23053441941738128662  200.0001068115234375  199.9451751708984375   
2      0.03476659581065177917  200.0000305175781250  199.9453735351562500   
3     -0.16300146281719207764  199.9997558593750000  199.9455566406250000   
4     -0.36278998851776123047  199.9992828369140625  199.9457702636718750   
...                       ...                   ...                   ...   
1619  34.31533050537109375000 -200.0000000000000000  202.7557830810546875   
1620  34.11040115356445312500 -200.0000000000000000  202.7222137451171875   
1621  33.90982818603515625000 -200.0000000000000000  202.6895599365234375   
1622  33.71352005004882812500 -200.0000000000000000  202.6577758789062500   
1623  33.52138519287109375000 -200.0000000000000000  202.6268463134765625   

                           u    

In [ ]:
from PIMNN_Phy_Bc import PIMNN_Phy_Bc
from plot import plot_predictions_vs_test, plot_test

# Train the model
model = PIMNN_Phy_Bc(df_train_n, df_freestream_n, df_aerofoil_n, mean_variance_dict, u_inlet, v_inlet, gamma_1, gamma_2, gamma_3)
# model.pre_train(51)
# print(f"Finished pre-Training.")
print(f"Started Training.")
model.train(101)
print(f"Finished Training.")

In [ ]:
# Prediction u_pred, v_pred, p_pred, nut_pred
nut_pred = model.predict(df_box_n, mean_variance_dict, u_inlet, v_inlet, gamma_1, gamma_2, gamma_3)

# Plotting
# plot_predictions_vs_test(df_box_n['x'].astype(float).values.flatten(), df_box_n['y'].astype(float).values.flatten(), u_pred, df_box_n['u'], 'u', 'PINN_u_Supervised')
# plot_predictions_vs_test(df_box_n['x'].astype(float).values.flatten(), df_box_n['y'].astype(float).values.flatten(), v_pred, df_box_n['v'], 'v', 'PINN_v_Supervised')
# plot_predictions_vs_test(df_box_n['x'].astype(float).values.flatten(), df_box_n['y'].astype(float).values.flatten(), p_pred, df_box_n['p'], 'p', 'PINN_p_Supervised')
plot_predictions_vs_test(df_box_n['x'].astype(float).values.flatten(), df_box_n['y'].astype(float).values.flatten(), nut_pred, df_box_n['nut'], 'nut', 'PINN_nut_Supervised')